In [97]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from IPython import get_ipython
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input
import os

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import torch.optim as optim
import missingno as msno

**Understanding the data**

In [ ]:
df = pd.read_csv("../datathon/dataset/product_data.csv")
outfit_data = pd.read_csv("../datathon/dataset/outfit_data.csv")
print(df.head(), outfit_data.head())

In [ ]:
df.describe()

In [ ]:
df.info()

**Preprocessing Data**

In [ ]:
# Creating new feature that is a list showcasing what outfits does each product appear in
outfits_agrupados = outfit_data.groupby('cod_modelo_color')['cod_outfit'].apply(list).reset_index()
outfits_agrupados.head()
df = pd.merge(df, outfits_agrupados, on='cod_modelo_color', how='left')
df.head()


In [ ]:
# save
df.isna().sum()



In [ ]:
msno.bar(df)

In [ ]:
df.isna().sum()


In [ ]:
msno.bar(df)


In [106]:
categorical_columns = ["cod_color_code","des_color_specification_esp","des_agrup_color_eng","des_sex","des_age","des_line","des_fabric","des_product_category","des_product_aggregated_family","des_product_family","des_product_type"]

for column in categorical_columns:
    df[column] = df[column].astype('category')

In [ ]:
# contamos cuantos valores unicos hay en cada columna
for x in categorical_columns:
    print(df[x].nunique())


Podemos ver que existen un cojon de categorias

In [ ]:
plt.figure(figsize=(20, 15))
plotnumber = 1

for column in df:
    if plotnumber <= 12 and pd.api.types.is_categorical_dtype(df[column]):
        ax = plt.subplot(3, 4, plotnumber)
        sns.countplot(x=df[column])
        plt.xlabel(column)
        plt.xticks(rotation=45)
        plotnumber += 1

plt.tight_layout()
plt.show()


In [ ]:
# Step 1: Initialize ResNet50 (pre-trained on ImageNet, remove the final classification layer)
# Load the pre-trained ResNet model with locally saved weights
resnet_model = ResNet50(weights='../model/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, pooling='avg')

# Step 2: Define function to process each image
def process_image(img_path):
    img = image.load_img(img_path, target_size=(239, 334))  # ResNet50 expects 224x224 input
    img_data = image.img_to_array(img)                      # Convert to array
    img_data = np.expand_dims(img_data, axis=0)             # Expand dimensions to match model input
    img_data = preprocess_input(img_data)                   # Preprocess (scale, normalize, etc.)
    return img_data

# Step 3: Define function to extract image embeddings
def extract_image_embedding(img_path):
    img_data = process_image(img_path)
    embedding = resnet_model.predict(img_data)  # Pass image through the pre-trained ResNet
    return embedding.flatten()                  # Flatten the output to get the feature vector

# Step 4: Load image paths and extract embeddings
image_folder = '/datathon/images'
image_embeddings = []

# Iterate through all image filenames in the dataset

for img_filename in df['des_filename']:
    img_path = os.path.join(image_folder, img_filename)
    embedding = extract_image_embedding(img_path)
    image_embeddings.append(embedding)

# Convert list of embeddings into a NumPy array
image_embeddings = np.array(image_embeddings)

In [ ]:
print('hey')

In [ ]:

def guardar_outputs(archivo_salida, caracter_inicial):
    # Obtener el historial del entorno de ejecución
    ipython_historial = get_ipython().history_manager
    outputs = get_ipython().user_ns['Out']
    
    # Abrir el archivo donde se guardarán los outputs
    with open(archivo_salida, 'w') as f:
        for cell_number, output in outputs.items():
            # Verificar si el número de celda empieza con el caracter especificado
            if str(cell_number).startswith(caracter_inicial):
                f.write(f"Output de la celda [{cell_number}]:\n")
                f.write(str(output))
                f.write("\n" + "="*50 + "\n")
    
    print(f"Outputs guardados en {archivo_salida}")

# Llamada a la función
guardar_outputs("outputs_notebook.txt","# save")